# Week 2 - Preprocessing, part 2

# 1. Lesson: None

# 2. Weekly graph question

The Storytelling With Data book mentions planning on a "Who, What, and How" for your data story.  Write down a possible Who, What, and How for your data, using the ideas in the book.

*DATASET*
Flight delay dataset

*WHO* 
Airline Operations Leaders who want to improve the flight experience for customers and reduce operational costs.

*WHAT*
They want to know which factors contribute flight delays in order to optimize the deployment of resources and communication to customers.

*HOW*
The Flight Delay dataset can help idenitfy delay factors by route, weather, temperature, airline, etc.

# 3. Homework - work with your own data

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [19]:
flight_df = pd.read_csv("DelayData.csv", delimiter=",")
flight_df = flight_df.head(1000)


This week, you will do the same types of exercises as last week, but you should use your chosen datasets that someone in your class found last semester. (They likely will not be the particular datasets that you found yourself.)

### Here are some types of analysis you can do  Use Google, documentation, and ChatGPT to help you:

- Summarize the datasets using info() and describe()

- Are there any duplicate rows?

- Are there any duplicate values in a given column (when this would be inappropriate?)

- What are the mean, median, and mode of each column?

- Are there any missing or null values?

    - Do you want to fill in the missing value with a mean value?  A value of your choice?  Remove that row?

- Identify any other inconsistent data (e.g. someone seems to be taking an action before they are born.)

- Encode any categorical variables (e.g. with one-hot encoding.)

### Conclusions:

- Are the data usable?  If not, find some new data!

- Do you need to modify or correct the data in some way?

- Is there any class imbalance?  (Categories that have many more items than other categories).

In [20]:
flight_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 61 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   depdelay                 1000 non-null   int64  
 1   arrdelay                 997 non-null    float64
 2   scheduleddepartdatetime  1000 non-null   object 
 3   origin                   1000 non-null   object 
 4   dest                     1000 non-null   object 
 5   uniquecarrier            1000 non-null   object 
 6   marketshareorigin        1000 non-null   float64
 7   marketsharedest          1000 non-null   float64
 8   hhiorigin                1000 non-null   float64
 9   hhidest                  1000 non-null   float64
 10  nonhubairportorigin      1000 non-null   int64  
 11  smallhubairportorigin    1000 non-null   int64  
 12  mediumhubairportorigin   1000 non-null   int64  
 13  largehubairportorigin    1000 non-null   int64  
 14  nonhubairportdest        

In [21]:
flight_df.describe()

,depdelay,arrdelay,marketshareorigin,marketsharedest,hhiorigin,hhidest,nonhubairportorigin,smallhubairportorigin,mediumhubairportorigin,largehubairportorigin,...,windgustdummy,windgustspeed,raindummy,raintracedummy,snowdummy,snowtracedummy,originmetropop,originmetrogdppercapita,destmetropop,destmetrogdppercapita
count,1000.000000,997.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,...,1000.000000,995.000000,1000.000000,1000.000000,1000.000000,1000.000000,1.000000e+03,1000.000000,1.000000e+03,1000.000000
mean,6.349000,5.802407,0.298460,0.302015,0.300853,0.304196,0.235000,0.172000,0.188000,0.405000,...,0.176000,2.421794,0.087000,0.050000,0.016000,0.082000,4.351151e+06,48036.239709,4.610348e+06,48274.695518
std,22.952865,27.128729,0.226934,0.233463,0.141235,0.159431,0.424211,0.377569,0.390908,0.491138,...,0.381011,7.181684,0.281976,0.218054,0.125538,0.274502,4.590279e+06,8863.113115,4.853023e+06,8618.005454
min,-16.000000,-45.000000,0.002101,0.000948,0.099479,0.099479,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.865900e+04,17239.758000,8.168100e+04,17546.180000
25%,-4.000000,-8.000000,0.092826,0.095781,0.213558,0.205015,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.391019e+06,43024.180000,1.471706e+06,43823.492000
50%,0.000000,-1.000000,0.267423,0.265995,0.298175,0.298175,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.935672e+06,49081.773000,3.106569e+06,49081.773000
75%,5.000000,11.000000,0.491505,0.478751,0.341763,0.341763,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.190444e+06,53535.344000,5.190444e+06,53535.344000
max,214.000000,224.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,40.000000,1.000000,1.000000,1.000000,1.000000,1.874743e+07,92839.820000,1.874743e+07,92839.820000


In [22]:
duplicate_rows = flight_df[flight_df.duplicated()]
print(duplicate_rows.shape[0])


0


In [23]:
print(flight_df.isna().any(axis=1).sum())

8


# 4. Storytelling With Data graph

Just like last week: choose any graph in the Introduction of Storytelling With Data. Use matplotlib to reproduce it in a rough way. I don't expect you to spend an enormous amount of time on this; I understand that you likely will not have time to re-create every feature of the graph. However, if you're excited about learning to use matplotlib, this is a good way to do that. You don't have to duplicate the exact values on the graph; just the same rough shape will be enough.  If you don't feel comfortable using matplotlib yet, do the best you can and write down what you tried or what Google searches you did to find the answers.